This notebook demonstrates unit conversion in the profile viewer

In [ ]:
import numpy as np
from astropy.wcs import WCS
import glue_jupyter as gj
from glue.core import Data
from glue.plugins.wcs_autolinking.wcs_autolinking import WCSLink

Set up two datasets with x-axes that use different spectral coordinates (which get auto-converted because of the WCSLink) and y-axes that use different temperature units (just to show how to add custom equivalencies)

In [ ]:
wcs1 = WCS(naxis=1)
wcs1.wcs.ctype = ['WAVE']
wcs1.wcs.crval = [10]
wcs1.wcs.cdelt = [3]
wcs1.wcs.crpix = [1]
wcs1.wcs.cunit = ['cm']

d1 = Data(temp1=np.random.random(10))
d1.get_component('temp1').units = 'deg_C'
d1.coords = wcs1

In [ ]:
wcs1 = WCS(naxis=1)
wcs1.wcs.ctype = ['FREQ']
wcs1.wcs.crval = [1000]
wcs1.wcs.cdelt = [1000]
wcs1.wcs.crpix = [1]
wcs1.wcs.cunit = ['MHz']

d2 = Data(temp2=np.random.random(10) + 273)
d2.get_component('temp2').units = 'K'
d2.coords = wcs1

Set up a custom unit conversion class with equivalencies:

In [ ]:
from astropy import units as u
from glue.core.units import unit_converter

@unit_converter('custom-astropy')
class UnitConverterWithTemperature:

    def equivalent_units(self, data, cid, units):
        equivalencies = u.temperature() if 'temp' in cid.label.lower() else None
        return map(str, u.Unit(units).find_equivalent_units(include_prefix_units=True, equivalencies=equivalencies))

    def to_unit(self, data, cid, values, original_units, target_units):
        equivalencies = u.temperature() if 'temp' in cid.label.lower() else None
        return (values * u.Unit(original_units)).to_value(target_units, equivalencies=equivalencies)

Use the converter class defined above

In [ ]:
from glue.config import settings
settings.UNIT_CONVERTER = 'custom-astropy'

In [ ]:
app = gj.jglue(spectrum1=d1, spectrum2=d2)

In [ ]:
app.output

In [ ]:
app.data_collection.add_link(WCSLink(d1, d2))

In [ ]:
app.data_collection

In [ ]:
s = app.profile1d(data=d1)
s.add_data(d2)

In [ ]:
s.state.x_display_unit = 'cm'
s.state.y_display_unit = 'K'